# Weight clustering in Keras example

## Overview

Welcome to the end-to-end example for *weight clustering*, part of the TensorFlow Model Optimization Toolkit.

### Other pages

For an introduction to what weight clustering is and to determine if you should use it (including what's supported), see the [overview page](https://www.tensorflow.org/model_optimization/guide/clustering).

To quickly find the APIs you need for your use case (beyond fully clustering a model with 16 clusters), see the [comprehensive guide](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_comprehensive_guide).

### Contents

In the tutorial, you will:

1. Train a `tf.keras` model for the MNIST dataset from scratch.
2. Fine-tune the model by applying the weight clustering API and see the accuracy.
3. Create a 6x smaller TF and TFLite models from clustering.
4. Create a 8x smaller TFLite model from combining weight clustering and post-training quantization.
5. See the persistence of accuracy from TF to TFLite.

## Setup

You can run this Jupyter Notebook in your local [virtualenv](https://www.tensorflow.org/install/pip?lang=python3#2.-create-a-virtual-environment-recommended) or [colab](https://colab.sandbox.google.com/). For details of setting up dependencies, please refer to the [installation guide](https://www.tensorflow.org/model_optimization/guide/install). 

In [2]:
! pip install -q tensorflow-model-optimization

In [3]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import tempfile
import zipfile
import os

## Train a tf.keras model for MNIST without clustering

In [4]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images  = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    validation_split=0.1,
    epochs=10
)


    8192/11490434 [..............................] - ETA: 0s


11490434/11490434 [==============================] - 0s 0us/step


2023-05-26 11:15:37.056973: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/10



   1/1688 [..............................] - ETA: 14:18 - loss: 2.3417 - accuracy: 0.0625

...


1635/1688 [============================>.] - ETA: 0s - loss: 0.0353 - accuracy: 0.9898


1649/1688 [============================>.] - ETA: 0s - loss: 0.0352 - accuracy: 0.9898


1663/1688 [============================>.] - ETA: 0s - loss: 0.0352 - accuracy: 0.9898


1677/1688 [============================>.] - ETA: 0s - loss: 0.0354 - accuracy: 0.9897


1688/1688 [==============================] - 7s 4ms/step - loss: 0.0354 - accuracy: 0.9897 - val_loss: 0.0617 - val_accuracy: 0.9832


### Evaluate the baseline model and save it for later usage

In [5]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Baseline test accuracy: 0.9790999889373779
Saving model to:  /tmpfs/tmp/tmp9jq1ksoy.h5


/tmpfs/tmp/ipykernel_14416/1025833175.py:8: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)


## Fine-tune the pre-trained model with clustering

Apply the `cluster_weights()` API to a whole pre-trained model to demonstrate its effectiveness in reducing the model size after applying zip while keeping decent accuracy. For how best to balance the accuracy and compression rate for your use case, please refer to the per layer example in the [comprehensive guide](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_comprehensive_guide).


### Define the model and apply the clustering API

Before you pass the model to the clustering API, make sure it is trained and shows some acceptable accuracy.

In [6]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

Model: "sequential"


_________________________________________________________________


 Layer (type)                Output Shape              Param #   


 cluster_reshape (ClusterWe  (None, 28, 28, 1)         0         


 ights)                                                          


...

 hts)                                                            


Total params: 40830 (239.13 KB)


Trainable params: 20442 (79.85 KB)


Non-trainable params: 20388 (159.28 KB)


_________________________________________________________________


### Fine-tune the model and evaluate the accuracy against baseline

Fine-tune the model with clustering for 1 epoch.

In [7]:
# Fine-tune model
clustered_model.fit(
  train_images,
  train_labels,
  batch_size=500,
  epochs=1,
  validation_split=0.1)


  1/108 [..............................] - ETA: 1:12 - loss: 0.0421 - accuracy: 0.9920


  5/108 [>.............................] - ETA: 1s - loss: 0.0442 - accuracy: 0.9860  


 10/108 [=>............................] - ETA: 1s - loss: 0.0447 - accuracy: 0.9858


 15/108 [===>..........................] - ETA: 1s - loss: 0.0438 - accuracy: 0.9856


 19/108 [====>.........................] - ETA: 1s - loss: 0.0417 - accuracy: 0.9866

...


 89/108 [=======================>......] - ETA: 0s - loss: 0.0407 - accuracy: 0.9868


 95/108 [=========================>....] - ETA: 0s - loss: 0.0405 - accuracy: 0.9869


100/108 [==========================>...] - ETA: 0s - loss: 0.0404 - accuracy: 0.9868


105/108 [============================>.] - ETA: 0s - loss: 0.0402 - accuracy: 0.9869


108/108 [==============================] - 2s 13ms/step - loss: 0.0402 - accuracy: 0.9868 - val_loss: 0.0696 - val_accuracy: 0.9820


For this example, there is minimal loss in test accuracy after clustering, compared to the baseline.

In [8]:
_, clustered_model_accuracy = clustered_model.evaluate(
  test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)

Baseline test accuracy: 0.9790999889373779
Clustered test accuracy: 0.9779999852180481


## Create **6x** smaller models from clustering

Both `strip_clustering` and applying a standard compression algorithm (e.g. via gzip) are necessary to see the compression benefits of clustering. 

First, create a compressible model for TensorFlow. Here, `strip_clustering` removes all variables (e.g. `tf.Variable` for storing the cluster centroids and the indices) that clustering only needs during training, which would otherwise add to model size during inference.

In [9]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
print('Saving clustered model to: ', clustered_keras_file)
tf.keras.models.save_model(final_model, clustered_keras_file, 
                           include_optimizer=False)

Saving clustered model to:  /tmpfs/tmp/tmpx4u1ju_5.h5


/tmpfs/tmp/ipykernel_14416/2908804151.py:5: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(final_model, clustered_keras_file,


Then, create compressible models for TFLite. You can convert the clustered model to a format that's runnable on your targeted backend. TensorFlow Lite is an example you can use to deploy to mobile devices.

In [10]:
clustered_tflite_file = '/tmp/clustered_mnist.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_clustered_model = converter.convert()
with open(clustered_tflite_file, 'wb') as f:
  f.write(tflite_clustered_model)
print('Saved clustered TFLite model to:', clustered_tflite_file)

INFO:tensorflow:Assets written to: /tmpfs/tmp/tmpt21y56f3/assets


INFO:tensorflow:Assets written to: /tmpfs/tmp/tmpt21y56f3/assets


2023-05-26 11:16:50.733209: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-26 11:16:50.733245: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.


Saved clustered TFLite model to: /tmp/clustered_mnist.tflite


Define a helper function to actually compress the models via gzip and measure the zipped size.

In [11]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

Compare and see that the models are **6x** smaller from clustering

In [12]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size(clustered_tflite_file)))

Size of gzipped baseline Keras model: 78169.00 bytes
Size of gzipped clustered Keras model: 12649.00 bytes
Size of gzipped clustered TFlite model: 12230.00 bytes


## Create an **8x** smaller TFLite model from combining weight clustering and post-training quantization

You can apply post-training quantization to the clustered model for additional benefits.

In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

_, quantized_and_clustered_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_clustered_tflite_file, 'wb') as f:
  f.write(tflite_quant_model)

print('Saved quantized and clustered TFLite model to:', quantized_and_clustered_tflite_file)
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_clustered_tflite_file)))

INFO:tensorflow:Assets written to: /tmpfs/tmp/tmpuk3uupzj/assets


INFO:tensorflow:Assets written to: /tmpfs/tmp/tmpuk3uupzj/assets


2023-05-26 11:16:51.350625: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-26 11:16:51.350663: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.


Saved quantized and clustered TFLite model to: /tmpfs/tmp/tmpnwe0hp0u.tflite
Size of gzipped baseline Keras model: 78169.00 bytes
Size of gzipped clustered and quantized TFlite model: 9466.00 bytes


## See the persistence of accuracy from TF to TFLite

Define a helper function to evaluate the TFLite model on the test dataset.

In [14]:
def eval_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

You evaluate the model, which has been clustered and quantized, and then see the accuracy from TensorFlow persists to the TFLite backend.

In [15]:
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

test_accuracy = eval_model(interpreter)

print('Clustered and quantized TFLite test_accuracy:', test_accuracy)
print('Clustered TF test accuracy:', clustered_model_accuracy)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Evaluated on 0 results so far.
Evaluated on 1000 results so far.


Evaluated on 2000 results so far.
Evaluated on 3000 results so far.


Evaluated on 4000 results so far.
Evaluated on 5000 results so far.


Evaluated on 6000 results so far.
Evaluated on 7000 results so far.


Evaluated on 8000 results so far.
Evaluated on 9000 results so far.




Clustered and quantized TFLite test_accuracy: 0.9785
Clustered TF test accuracy: 0.9779999852180481


## Conclusion

In this tutorial, you saw how to create clustered models with the TensorFlow Model Optimization Toolkit API. More specifically, you've been through an end-to-end example for creating an 8x smaller model for MNIST with minimal accuracy difference. We encourage you to try this new capability, which can be particularly important for deployment in resource-constrained environments.
